In [6]:
import sys
# adding to the path variables the one folder higher (locally, not changing system variables)
sys.path.append("..")

# import needed functions
from scripts.processing import *

#read the full json
dfr = get_df('../data/yelp_academic_dataset_review.json')

# save full file as csv
dfr.to_csv('../data/review.csv')

# add year column and filter by it
dfr['year'] = pd.to_datetime(dfr.date).dt.year
df_two = dfr.query('2018 <= year <= 2019')

# save reduced df
df_two.to_csv('../data/review_1819.csv')

[nltk_data] Downloading package stopwords to /home/jerwin/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


FileNotFoundError: [Errno 2] No such file or directory: '../data/yelp_dataset/yelp_academic_dataset_review.json'